In [862]:
import pandas as pd #importamos la libreria "pandas" para hacer uso de df's
import nltk #importamos la libreria para el procesamiento
from nltk.corpus import stopwords #usamos la libreria stopwords para eliminarlas en el proceso
import math
import os
import numpy as np
stop_words = set(stopwords.words('english'))

In [863]:
#Clase que usaremos para las funciones (normalizacion)

class tukey_tools: #usamos una clase para tener todas las funciones almacenadas aqui

    def __init__(self):
        pass

    def to_lower(self, texto: str)->str: #convierte el texto a puras minusculas
        return texto.lower()
    
    def remove_special(self, text:str)->str: #quita los caracteres especiales, solo guarda letras y espacios
        result = ""
        for letter in text:
            if letter.isalpha() or letter.isspace():
                result += letter
            else:
                continue
        return result
    
    def tokenize(self, text: str)->list: #tokenizamos cada palabra del texto (guardamos una por una en una lista)
        tokens = list()
        j = 0
        for i in range(len(text)):
            if text[i] == ' ':
                tokens.append(text[j:i])
                j = i+1
            else:
                continue
        tokens.append(text[j:])
        return tokens
    
    def remove_stop_words(self, tokens: list)->list: #eliminamos las palabras que no aportan ningun significado
        no_stopwords = list()
        for i in tokens:
            if i not in stop_words and i != 'im':
                no_stopwords.append(i)
            else:
                continue
        return no_stopwords
    
    def words(self, frase: list, emocion: str)->dict: #crea un diccionario con palabras (tambien repetidas) de cada emocion 
        words = {} #diccionario vacio
        for i in range(len(emocion)): #usamos la longitud de la columna "emocion" para poder usar indices
            if emocion[i] not in words: #si la emocion ya se encuentra en las claves del diccionario entonces no se agrega
                words[emocion[i]] = [] 
            words[emocion[i]] += [frase[i]] #agrega las listas de la columna "frase" a la emocion que corresponda
        return words
    
    def count(self, palabras: dict)->dict: #cuenta cuantas veces se repite cada palabra
        for emotion in palabras.keys(): #recorremos cada emocion
            reps = {} #creamos un diccionario que se actualiza a vacio cada que cambia de emocion
            for word in palabras[emotion]: #recorremos las palabras de cada clave
                if word in reps: #si ya existe en el diccionario aumentamos su valor en uno
                    reps[word] = reps[word]+1
                else:
                    reps[word] = 1 #si no existe lo dejamos como =1 ya que solo esta una vez
            palabras[emotion] = reps #agregamos el diccionario a cada clave del diccionario principal
        return palabras
    
    def maxxx(self, palabras: dict)->dict: #funcion que extrae las 3 palabras mas comunes de cada clave
        maxx = [] #lista vacia
        for emotion in palabras.keys(): #se recorren las emociones
            maxx = [] #se actualiza la lista a vacia
            for i in range(3): #se recorre 3 veces para obtener 3 palabras distintas
                max_count = max(palabras[emotion], key = palabras[emotion].get) #se determina la palabra con mayor numero de repeticiones
                maxx.append(max_count) #agregamos la palabra a la lista
                del palabras[emotion][max_count] #borramos el valor del diccionario principal para obtener la segunda con mayor cantidad
            palabras[emotion] = maxx #asignamos las 3 palabras de la emocion al diccionario
        return palabras
    
    def emotions(self, emociones: str)->list: #devuelve las emociones sin repetir
        ems = []
        for emotion in emociones:
            if emotion not in ems:
                ems.append(emotion)
            else:
                continue
        return ems
    
    def inter_words(self, interseccion: dict, ems: list)->dict: #determina si hay palabras contenidas en dos o mas emociones y las almacena en su respectiva emocion
        inter = {}
        for emotion in interseccion.keys():
            reps = []
            ems.remove(emotion)
            for word in interseccion[emotion]:
                for em in ems:
                    if word in interseccion[em] and word not in reps:
                        reps.append(word)
            ems.append(emotion)
            inter[emotion] = reps
        return inter
        
    def max_corpus(self, palabras: list)->str: #determina la palabra mas mencionada en el corpus 
        words = {}
        for word in palabras:
            if word in words:
                words[word] = words[word]+1
            else:
                words[word] = 1
        most_rep = max(words, key = words.get)
        return most_rep
            
    def long(self, palabras: dict)->dict: #calcula la media del total de oraciones de cada emocion 
        suma = 0
        aux = 0
        promedio = 0
        long = {}
        for emotion in palabras.keys():
            aux = 0
            suma = 0
            for line in palabras[emotion]:
                aux += 1
                suma += len(line)
            promedio = suma/aux
            long[emotion] = promedio
        return long
    
    def exclusivas(self, ex: dict, ems: list)->dict:
        excl = {}
        w = []
        elims = []
        for emotion in ex.keys():
            w = []
            ems.remove(emotion)
            for word in ex[emotion]:
                for em in ems:
                    if word not in ex[em] and word not in elims:
                        if word not in w:
                            w.append(word)
                    elif word in ex[em] and word in w:
                        w.remove(word)
                        elims.append(word)
            excl[emotion] = w
            ems.append(emotion)
        return excl  
    
    def significado(self, interseccion: dict): #busca las palabras que se repiten en las emociones, ya que se usan en diferentes contextos
        reps = {}
        for emotion in interseccion.keys():
            for word in interseccion[emotion]:
                if word not in reps:
                    reps[word] = [emotion]
                elif word in reps and emotion not in reps[word]:
                    reps[word].append(emotion)
        return reps

    
    def tf_idf(self, tokens, vocab): #representacion numerica de los datos
        no_docs = len(tokens)
        # IDF
        idf = dict()
        for word in vocab:
            n_t = sum(1 for document in tokens if word in document)
            idf[word] = math.log(no_docs/(1+n_t))+1
        # TF-IDF
        tf_idf_matrix = []
        for doc in tokens:
            doc_len = len(doc)
            tf_idf_vector = []
            for word in vocab:
                tf = doc.count(word) / doc_len
                tf_idf_val = tf * idf[word]
                tf_idf_vector.append(tf_idf_val) 
            tf_idf_matrix.append(tf_idf_vector)
        return np.array(tf_idf_matrix)

In [864]:
#transformacion del archivo en formato txt a csv
import pandas as pd
txt = open("test.txt", "r", encoding="utf-8") #abrimos el archivo en forma de lectura
csv = open("examen.csv", "w", encoding="utf-8") #abrimos en archivo en modo escritura
csv.write('Frase;Emocion\n') #se imrpime esto (se busca que funcione como encabezado para el df)
for line in txt:
    csv.write(line) #copiamos el contenido de txt a csv 
txt.close() #cerramos ambos archivos
csv.close()

df = pd.read_csv('examen.csv', sep=';')#representamos el archivo en df, separando las columnas por ';'
print(df.head())

                                               Frase  Emocion
0  im feeling rather rotten so im not very ambiti...  sadness
1          im updating my blog because i feel shitty  sadness
2  i never make her separate from me because i do...  sadness
3  i left with my bouquet of red and yellow tulip...      joy
4    i was feeling a little vain when i did this one  sadness


In [865]:
#normalizacion de texto
herramientas = tukey_tools() #creamos una instancia para la clase 

#convertir todo a minusculas
df['Frase'] = df['Frase'].apply(herramientas.to_lower) 
df['Emocion'] = df['Emocion'].apply(herramientas.to_lower) 

#eliminamos caracteres especiales
df['Frase'] = df['Frase'].apply(herramientas.remove_special) 

#tokenizamos palabra por palabra
df['Frase'] = df['Frase'].apply(herramientas.tokenize) 

#eliminamos las stop words (no aportan ningun significado)
df['Frase'] = df['Frase'].apply(herramientas.remove_stop_words) 

print(df.head())


                                               Frase  Emocion
0        [feeling, rather, rotten, ambitious, right]  sadness
1                     [updating, blog, feel, shitty]  sadness
2  [never, make, separate, ever, want, feel, like...  sadness
3  [left, bouquet, red, yellow, tulips, arm, feel...      joy
4                       [feeling, little, vain, one]  sadness


In [866]:
#Representacion en tf-idf
tokens = df['Frase'].tolist()
vocab = list(set(word for line in tokens for word in line))
tf = herramientas.tf_idf(tokens, vocab)
emsss = [] 
emsss = df['Emocion'].tolist()
tfidf = pd.DataFrame(tf, columns = vocab)
tfidf.insert(0, 'Emocion', emsss)

# Guardar DataFrame a archivo CSV
tfidf.to_csv('tfidf_output.csv', index=False)



In [880]:
#Analisis
#1- Palabras mas comunes de cada emocion
conteo = {}
conteo =  herramientas.words(df['Frase'], df['Emocion']) #guardamos cada lista de la columna frase como valor de cada clave a la que pertenece (emocion)
for emocion in conteo.keys(): #recorremos las claves del diccionario (emociones)
    words = [] #creamos una lista donde se van  almacenar los elementos de las listas segun su clave
    for palabras in conteo[emocion]: #recorremos los valores de la emocion
        words.extend(palabras) #extendemos la lista, con el fin de tener una sola y no varias 
    conteo[emocion] = words #asignamos la anterior lista como valor de nuestra clave actual 
#print(conteo)comprobamos que de la salida esperada

conteo = herramientas.count(conteo) #llamamos a la funcion que se encarga del conteo para saber la cantidad de veces que se repite cada palabra
#print(conteo) comprobamos la salida

conteo = herramientas.maxxx(conteo) #llamamos a la funcion donde se obtienen las 3 palabras mas comunes de cada emocion
print("\n")

for emotion in conteo.keys():
    print(f"Palabras mas comunes de {emotion}: {conteo[emotion][:5]}")



Palabras mas comunes de sadness: ['feel', 'feeling', 'like']
Palabras mas comunes de joy: ['feel', 'feeling', 'like']
Palabras mas comunes de fear: ['feel', 'feeling', 'like']
Palabras mas comunes de anger: ['feel', 'feeling', 'like']
Palabras mas comunes de love: ['feel', 'feeling', 'like']
Palabras mas comunes de surprise: ['feel', 'feeling', 'like']


In [ ]:
#Analisis
#2- Interseccion de palabras de todas las emociones
interseccion = {} #creamos un diccionario vacio
interseccion = herramientas.words(df['Frase'], df['Emocion']) 
#convertimos todas las sublistas de cada emocion en una sola lista usando extend
for emocion in interseccion.keys(): 
    words = [] 
    for palabras in interseccion[emocion]: 
        words.extend(palabras) 
    interseccion[emocion] = words 
for emotion in interseccion.keys():
    print(f"{emotion}: {interseccion[emotion][:5]}") #solo se imprimen algunos para representar
print("\n")

ems = [] #creamos una lista vacia para almacenar las emociones
ems = herramientas.emotions(df['Emocion']) #llamamos a la funcion
#print(f"{ems}\n")#comprobamos la salida

inter = {}
inter = herramientas.inter_words(interseccion, ems) #llama a la funcion donde se determinan las intersecciones
for emotion in inter.keys():
    print(f"Las palabras de {emotion} que se intersectan con otras emociones, son: {inter[emotion][:5]}")



sadness: ['feeling', 'rather', 'rotten', 'ambitious', 'right']
joy: ['left', 'bouquet', 'red', 'yellow', 'tulips']
fear: ['cant', 'walk', 'shop', 'anywhere', 'feel']
anger: ['felt', 'anger', 'end', 'telephone', 'call']
love: ['find', 'odd', 'position', 'feeling', 'supportive']
surprise: ['feel', 'little', 'stunned', 'imagine', 'folks']


Las palabras de sadness que se intersectan con otras emociones, son: ['feeling', 'rather', 'right', 'updating', 'blog']
Las palabras de joy que se intersectan con otras emociones, son: ['left', 'red', 'yellow', 'feeling', 'slightly']
Las palabras de fear que se intersectan con otras emociones, son: ['cant', 'walk', 'feel', 'uncomfortable', 'particularly']
Las palabras de anger que se intersectan con otras emociones, son: ['felt', 'anger', 'end', 'call', 'feel']
Las palabras de love que se intersectan con otras emociones, son: ['find', 'odd', 'feeling', 'feel', 'like']
Las palabras de surprise que se intersectan con otras emociones, son: ['feel', 'littl

In [891]:
#Analisis
#3- Palabras con signficado diferente 
significa = {}
intersect = {}
ems_2 = []
ems_2 = herramientas.emotions(df['Emocion'])
intersect = herramientas.inter_words(interseccion, ems_2)
significa = herramientas.significado(intersect)

for word in significa.keys():
    i += 1
    if i < 5:
        print(f"Palabra: {word}: {significa[word]}")
    else:
        break


Palabra: feeling: ['sadness', 'joy', 'fear', 'anger', 'love', 'surprise']
Palabra: rather: ['sadness', 'joy', 'fear', 'anger']
Palabra: right: ['sadness', 'joy', 'fear', 'anger', 'love', 'surprise']
Palabra: updating: ['sadness', 'anger']


In [870]:
#Analisis
#4- Palabras mas frecuentes de todo el corpus
mayor_corpus = ""
lista = []
for line in df['Frase']:
    lista.extend(line) #guardamos todas las listas (de las oraciones) generadas en una sola para recorrerla
mayor_corpus = herramientas.max_corpus(lista) #llamamos a la funcion
print(f"La palabra mas usada en el corpus es: {mayor_corpus}")

La palabra mas usada en el corpus es: feel


In [871]:
#Analisis
#5- Comparar la longitud promedio de las oraciones por clase
palabras = {}
longitud = {}
palabras = herramientas.words(df['Frase'], df['Emocion']) #volvemos a usar el diccionario que nos devuelve cada oracion (ya procesada) para calcular el promedio
longitud = herramientas.long(palabras) #llamamos a la funcion para calcular la media
for emotion in longitud.keys():
    print(f"La longitud promedio de cada oracion correspondiente a {emotion} es: {longitud[emotion]}")
print("\n")
print(f"La emocion cuyas oraciones tienen el mayor promedio es {max(longitud, key = longitud.get)}, mientras que la de menor promedio es {min(longitud, key = longitud.get)}")

La longitud promedio de cada oracion correspondiente a sadness es: 9.08605851979346
La longitud promedio de cada oracion correspondiente a joy es: 9.130935251798562
La longitud promedio de cada oracion correspondiente a fear es: 8.700892857142858
La longitud promedio de cada oracion correspondiente a anger es: 9.374545454545455
La longitud promedio de cada oracion correspondiente a love es: 9.59748427672956
La longitud promedio de cada oracion correspondiente a surprise es: 9.318181818181818


La emocion cuyas oraciones tienen el mayor promedio es love, mientras que la de menor promedio es fear


In [894]:
#Analisis
#6- Palabras exclusivas de cada emocion
ems_dos = []
ems_dos = herramientas.emotions(df['Emocion'])
#print(ems_dos)

excl = {}
excl = herramientas.words(df['Frase'], df['Emocion']) 
for emocion in excl.keys(): 
    words = [] 
    for palabras in excl[emocion]: 
        words.extend(palabras) 
    excl[emocion] = words 
#print(excl) 

exclusiv = herramientas.exclusivas(excl, ems_dos)
for emotion in exclusiv.keys():
    print(f"Las palabras exclusivas de {emotion} son: {exclusiv[emotion][:5]}")



Las palabras exclusivas de sadness son: ['rotten', 'ambitious', 'shitty', 'separate', 'ashamed']
Las palabras exclusivas de joy son: ['bouquet', 'yellow', 'tulips', 'arm', 'clung']
Las palabras exclusivas de fear son: ['shop', 'anywhere', 'pay', 'deepens', 'invaded']
Las palabras exclusivas de anger son: ['telephone', 'jest', 'pre', 'menstrual', 'walrus']
Las palabras exclusivas de love son: ['position', 'supportive', 'naughty', 'border', 'foreigner']
Las palabras exclusivas de surprise son: ['stunned', 'shocked', 'handed', 'billiards', 'contractions']


In [873]:
#Conclusiones
#sadness: Las palabras relacionadas a esta emocion reflejan frustracion y desagrado, interpreto los sentimientos de alguien por haber perdido algo valioso o no cumplir un objetivo
#joy: Al menos en las palabras que describen esta emocion destacan las que hacen alucion a flores y a cosas sin un compromiso, puede ser una relacion joven que aun no es seria
#Fear: Las palabras correspondientes al miedo se caracterizan por la inseguridad o temerosidad, tambien algunas palabras mencionan acciones como pagar, se puede relacionar a no querer gastar dinero
#Anger: Representan frustracion acerca sobre vivencias personales o con 3ras personas
#Love: Son palabras que se refieren de muy buena forma a una persona, tal como ser solidario, se pueden relacionar mas que nada a una relacion y al romance
#Surprise: Algunas palabras resaltan el impacto al recirbir una gran sorpresa, un ejemplo es "shocked" o a entregar o recibir cosas

#insights generales: desde mi perspectiva, fue un ejercicio muy interesante y divertido, realmente se nota la forma en la que ayuda la normalizacion de los datos,
        #se puede ver como al inicio hay muchas palabras que no aportan ninguna importancia o significado pero despues de este proceso, el numero
        #decrementa considerablemente, permitiendo una mayor facilidad en la manipulacion de estos datos, ya sea para su analisis como en este caso,
        #tambien nos permitio conocer las palabras que toman un diferente significado dependiendo su contexto, como hay palabras caracteristicas y exclusivas
        #para cada emocion, realmente fue muy agradable realizar esta prueba.